In [68]:
import numpy as np
from math import log
class BaseEstimator(object):
    """
    Base class for estimators in pgmpy; `ParameterEstimator`,
    `StructureEstimator` and `StructureScore` derive from this class.

    Parameters
    ----------
    data: pandas DataFrame object
        object where each column represents one variable.
        (If some values in the data are missing the data cells should be set to `numpy.NaN`.
        Note that pandas converts each column containing `numpy.NaN`s to dtype `float`.)

    state_names: dict (optional)
        A dict indicating, for each variable, the discrete set of states (or values)
        that the variable can take. If unspecified, the observed values in the data set
        are taken to be the only possible states.
    """

    def __init__(self, data=None, state_names=None):
        self.data = data
        # data can be None in the case when learning structure from
        # independence conditions. Look into PC.py.
        if self.data is not None:
            self.variables = list(data.columns.values)

            if not isinstance(state_names, dict):
                self.state_names = {
                    var: self._collect_state_names(var) for var in self.variables
                }
            else:
                self.state_names = dict()
                for var in self.variables:
                    if var in state_names:
                        if not set(self._collect_state_names(var)) <= set(
                            state_names[var]
                        ):
                            raise ValueError(
                                f"Data contains unexpected states for variable: {var}."
                            )
                        self.state_names[var] = state_names[var]
                    else:
                        self.state_names[var] = self._collect_state_names(var)

    def _collect_state_names(self, variable):
        "Return a list of states that the variable takes in the data."
        states = sorted(list(self.data.loc[:, variable].dropna().unique()))
        return states

    def state_counts(
        self,
        variable,
        parents=[],
        weighted=False,
        reindex=True,
    ):
        """
        Return counts how often each state of 'variable' occurred in the data.
        If a list of parents is provided, counting is done conditionally
        for each state configuration of the parents.

        Parameters
        ----------
        variable: string
            Name of the variable for which the state count is to be done.

        parents: list
            Optional list of variable parents, if conditional counting is desired.
            Order of parents in list is reflected in the returned DataFrame

        weighted: bool
            If True, data must have a `_weight` column specifying the weight of the
            datapoint (row). If False, each datapoint has a weight of `1`.

        reindex: bool
            If True, returns a data frame with all possible parents state combinations
            as the columns. If False, drops the state combinations which are not
            present in the data.

        Returns
        -------
        state_counts: pandas.DataFrame
            Table with state counts for 'variable'

        Examples
        --------
        >>> import pandas as pd
        >>> from pgmpy.estimators import BaseEstimator
        >>> data = pd.DataFrame(data={'A': ['a1', 'a1', 'a2'],
                                      'B': ['b1', 'b2', 'b1'],
                                      'C': ['c1', 'c1', 'c2']})
        >>> estimator = BaseEstimator(data)
        >>> estimator.state_counts('A')
            A
        a1  2
        a2  1
        >>> estimator.state_counts('C', parents=['A', 'B'])
        A  a1      a2
        B  b1  b2  b1  b2
        C
        c1  1   1   0   0
        c2  0   0   1   0
        >>> estimator.state_counts('C', parents=['A'])
        A    a1   a2
        C
        c1  2.0  0.0
        c2  0.0  1.0
        """
        parents = list(parents)

        if weighted and ("_weight" not in self.data.columns):
            raise ValueError("data must contain a `_weight` column if weighted=True")

        if not parents:
            # count how often each state of 'variable' occurred
            if weighted:
                state_count_data = self.data.groupby([variable])["_weight"].sum()
            else:
                state_count_data = self.data.loc[:, variable].value_counts()

            state_counts = (
                state_count_data.reindex(self.state_names[variable])
                .fillna(0)
                .to_frame()
            )

        else:
            parents_states = [self.state_names[parent] for parent in parents]
            # count how often each state of 'variable' occurred, conditional on parents' states
            if weighted:
              state_count_data = (
                  self.data.groupby([variable] + parents)["_weight"]
                  .sum()
                  .unstack(parents)
              )


            else:
              state_count_data = (
                  self.data.groupby([variable] + parents).size().unstack(parents)
              )

            if not isinstance(state_count_data.columns, pd.MultiIndex):
              state_count_data.columns = pd.MultiIndex.from_arrays(
                  [state_count_data.columns]
              )

            if reindex:
              # reindex rows & columns to sort them and to add missing ones
              # missing row    = some state of 'variable' did not occur in data
              # missing column = some state configuration of current 'variable's parents
              #                  did not occur in data
              row_index = self.state_names[variable]
              column_index = pd.MultiIndex.from_product(parents_states, names=parents)
              state_counts = state_count_data.reindex(
                  index=row_index, columns=column_index
              ).fillna(0)
            else:
              state_counts = state_count_data.fillna(0)

        return state_counts
    def local_score(self, variable, parents):
        'Computes a score that measures how much a \
        given variable is "influenced" by a given list of potential parents.'

        var_states = self.state_names[variable]
        var_cardinality = len(var_states)
        parents = list(parents)
        state_counts = self.state_counts(variable, parents, reindex=False)
        sample_size = len(self.data)
        num_parents_states = np.prod([len(self.state_names[var]) for var in parents])

        counts = np.asarray(state_counts)
        log_likelihoods = np.zeros_like(counts, dtype=float)
        # Compute the log-counts
        np.log(counts, out=log_likelihoods, where=counts > 0)
        # Compute the log-conditional sample size
        log_conditionals = np.sum(counts, axis=0, dtype=float)
        np.log(log_conditionals, out=log_conditionals, where=log_conditionals > 0)
        # Compute the log-likelihoods
        log_likelihoods -= log_conditionals
        log_likelihoods *= counts

        score = np.sum(log_likelihoods)

        score -= 0.5 * log(sample_size) * num_parents_states * (var_cardinality - 1)

        return score


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [69]:
def is_dag(adjacency_matrix):
  """Checks if an adjacency matrix is nilpotent.

  Args:
    adjacency_matrix: A numpy array representing the adjacency matrix.

  Returns:
    True if the adjacency matrix is nilpotent, False otherwise.
  """

  for i in range(len(adjacency_matrix)):
    power = np.linalg.matrix_power(adjacency_matrix, i + 1)
    if np.trace(power) != 0:
      return False

  return True

def calculate_bic_score(graph, base_estimate):
  nodes = list(base_estimate.state_names.keys())

  num_nodes = len(nodes)
  bic_score = 0
  for k in range(num_nodes):
    parents = np.nonzero(graph[:,k])[0]
    parents = [nodes[k] for k in parents]
    if nodes[k] in parents:
      parents.remove(nodes[k])
    bic_score += base_estimate.local_score(nodes[k], parents)
  return bic_score

In [70]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Asia.csv")
data = data.drop('Unnamed: 0', axis=1)
base_estimate = BaseEstimator(data)
graph = np.array([[0, 0, 0, 0, 0, 0, 0, 0],[0., 0., 0., 1., 0., 0., 0., 0.], [0., 0., 0., 0., 1., 1., 0., 1.], [0., 0., 0., 0., 0., 1., 0., 0.],
 [0., 1., 0., 1., 0., 0., 0., 0.],[0., 0., 0., 0., 0., 0., 1., 0.],[0., 0., 0., 0., 0., 0., 0., 0.], [0., 1., 0., 1., 1., 0., 0., 0.]])

print(calculate_bic_score(graph, base_estimate))

-11141.386647046336


In [71]:
import gym
from gym import spaces
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
# Define the Gym environment for the causal graph problem
class CausalGraphEnv(gym.Env):
    def __init__(self, n):
        super(CausalGraphEnv, self).__init__()
        self.n = n
        self.action_space = spaces.Discrete(n * n * 2)  # Two actions for each element in the matrix
        self.observation_space = spaces.Box(low=0, high=1, shape=(n, n), dtype=np.uint8)
        self.min_bic = 99999999
        self.best_state = np.zeros((self.n, self.n), dtype=np.uint8)

    def reset(self):
        self.state = np.zeros((self.n, self.n), dtype=np.uint8)
        return self.state

    def step(self, action, base_estimate):
        print("actions: ", action)
        i, j = divmod(action // 2, self.n)  # Determine the matrix element to change
        toggle = action % 2  # Determine whether to set or unset the edge

        # Save the current state to revert back if the action is invalid
        original_state = np.copy(self.state)
        # Apply the action
        self.state[i, j] = 1-self.state[i,j] if toggle == 1 else 0

        if is_dag(self.state):
            # Calculate the BIC score and determine the reward
            new_bic_score = calculate_bic_score(self.state, base_estimate)
            bic_score = - new_bic_score  # Reward is the negative BIC score
            done = False  # We can define the termination condition based on the problem
        else:
            # Revert to the original state as the action results in a non-DAG
            self.state = original_state
            bic_score = -1  # Penalize the action that results in a non-DAG
            done = True  # Optionally end the episode
        if bic_score < self.min_bic and bic_score > 0:
            self.best_state = np.copy(self.state)
            self.min_bic = bic_score
        return self.state, bic_score, done, {}

    def render(self, mode='console'):
        if mode != 'console':
            raise NotImplementedError()
        print(self.state)
    def result(self):
        return self.best_state

# Define the Q-network using PyTorch
class QNetwork(nn.Module):
    def __init__(self, n):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(n * n, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, n * n * 2)  # Output size: n*n*2 (for each element in the matrix, two possible actions)

    def forward(self, x):
        x = torch.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [75]:
epsilon = 0.5
class QLearningAgent:
    def __init__(self, env, q_network):
        self.env = env
        self.q_network = q_network
        self.optimizer = optim.Adam(q_network.parameters(), lr=0.001)

    def train(self,base_estimate, episodes=1000, gamma=0.99):
        for episode in range(episodes):
            state = self.env.reset()
            total_reward = 0
            done = False
            while not done:

                # Convert state to tensor for PyTorch
                state_tensor = torch.FloatTensor(state).unsqueeze(0)

                # Epsilon-greedy strategy for action selection
                if random.random() < epsilon:
                    action = self.env.action_space.sample()
                else:
                    q_values = self.q_network(state_tensor)
                    action = torch.argmax(q_values).item()

                # Take action and observe new state and reward
                new_state, bic_score, done, _ = self.env.step(action, base_estimate)

                # Calculate BIC score based change as reward
                old_bic_score = calculate_bic_score(state, base_estimate)
                new_bic_score = calculate_bic_score(new_state, base_estimate)

                reward =  new_bic_score - old_bic_score
                # Convert new state to tensor
                new_state_tensor = torch.FloatTensor(new_state).unsqueeze(0)

                # Calculate target Q-value
                target_q_value = reward + gamma * torch.max(self.q_network(new_state_tensor)).item()
                target_q_value = torch.tensor(target_q_value, dtype = torch.float)
                # Get current Q-value

                current_q_value = self.q_network(state_tensor)[action]
                #print(nn.functional.mse_loss(float(current_q_value), target_q_value))
                # Calculate loss
                print("Current_q_value", current_q_value)
                loss = nn.functional.mse_loss(current_q_value, target_q_value)


                # Backpropagation
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                state = np.copy(new_state)
                total_reward += reward

            print(f"Episode {episode}, Total Reward: {total_reward}")

# Example usage
env = CausalGraphEnv(n=8)
q_network = QNetwork(n=8)
agent = QLearningAgent(env, q_network)
agent.train(base_estimate ,episodes=100)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


actions:  119
Current_q_value tensor(0.2478, grad_fn=<SelectBackward0>)
actions:  119
Current_q_value tensor(0.2753, grad_fn=<SelectBackward0>)
actions:  126
Current_q_value tensor(0.1916, grad_fn=<SelectBackward0>)
actions:  109
Current_q_value tensor(0.1389, grad_fn=<SelectBackward0>)
Episode 0, Total Reward: -81.8364848624733
actions:  35
Current_q_value tensor(-0.1335, grad_fn=<SelectBackward0>)
actions:  119
Current_q_value tensor(0.2608, grad_fn=<SelectBackward0>)
actions:  60
Current_q_value tensor(0.1571, grad_fn=<SelectBackward0>)
actions:  119
Current_q_value tensor(0.2644, grad_fn=<SelectBackward0>)
actions:  62
Current_q_value tensor(-0.1208, grad_fn=<SelectBackward0>)
actions:  66
Current_q_value tensor(0.1173, grad_fn=<SelectBackward0>)
actions:  116
Current_q_value tensor(-0.1893, grad_fn=<SelectBackward0>)
actions:  119
Current_q_value tensor(0.2632, grad_fn=<SelectBackward0>)
actions:  74
Current_q_value tensor(-0.2260, grad_fn=<SelectBackward0>)
actions:  59
Current_q

In [76]:
print(env.best_state, env.min_bic)
print(calculate_bic_score(env.best_state, base_estimate))

[[0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 1 0 1 0 0 0 1]
 [0 0 0 0 1 1 0 0]
 [0 1 0 1 1 0 0 0]] 11401.946490471273
-11401.946490471273
